In [1]:
import numpy as np
from matplotlib import pyplot as plt
import ipdb
np.random.seed(76) # Felt Lucky :P 

image_height = 32
image_width = 32
image_depth = 1

filter_shape = 5



In [2]:

def relu(data):                     #Activation function
    return np.maximum(data, 0)
    

def convolution(image, filter):     #Convolution function

    filter_height, filter_width, filter_depth = filter.shape
    image_height, image_width, image_depth = image.shape

    output_height = image_height - filter_height + 1
    output_width = image_width - filter_width + 1
    output_depth = filter_depth

    output = np.zeros((output_height, output_width, output_depth))

    for y in range(output_height):
        for x in range(output_width):
            for z in range(output_depth):
            
                image_patch = image[y:y+filter_height, x:x+filter_width, :] 
                filter_of_iterest = filter[:, :, z]

                dot_product = np.dot( filter_of_iterest, image_patch.T)
                output[y, x, z] = np.sum(dot_product )

    return output

def maxpool(image, kernel_shape):     #MaxPool function


    image_height, image_width, image_depth = image.shape

    output_height = int((image_height - kernel_shape)/kernel_shape) + 1
    output_width = int((image_width - kernel_shape)/kernel_shape) + 1
    output_depth = image_depth

    output = np.zeros((output_height, output_width, output_depth))

    for y in range(0, output_height, kernel_shape):
        for x in range(0, output_width, kernel_shape):
            for z in range(output_depth):
                
                image_patch = image[y:y+kernel_shape, x:x+kernel_shape, z] 
                output[y, x, z] = np.max(image_patch )
      

    return output


def softmax(z):                         #Softmax function

    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting)
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div


def linear(x, weights, bias):          #Linear function
  
    out = np.dot(x,weights) + bias
    return out
    


In [3]:
                            ###########   Data Allocation ###########

# Have used random data for now, will change it to the actual data later

image = np.random.rand(image_height, image_width, image_depth)  #Maybe use image of Dustin :P
filter_depths = [8, 16]
filter1_weights = np.random.rand(filter_shape,filter_shape, filter_depths[0])   # Change the filter size here
filter1_bias = np.random.rand(image_height - filter_shape + 1, image_width - filter_shape + 1, filter_depths[0])
filter2_weights = np.random.rand(filter_shape,filter_shape,filter_depths[1])
bias_dim = int((image_height + 1 - 3*filter_shape)/2) + 1
filter2_bias = np.random.rand(bias_dim, bias_dim, filter_depths[1])
linear_input_dim = int((bias_dim - 2)/2) +1
linear1_weights = np.random.rand(linear_input_dim * linear_input_dim * filter_depths[1], 128)
linear1_bias = np.random.rand(128)
linear2_weights = np.random.rand(128, 64)      
linear2_bias = np.random.rand(64)
linear3_weights = np.random.rand(64, 10)
linear3_bias = np.random.rand(10)        # 10 classes (we can increase depending on the boards config)

# TODO:
# add Batch size for inference

In [4]:
 ############################ Convoltion and MaxPool Layers ############################
conv_out1 = relu(convolution(image, filter1_weights) + filter1_bias)        # 32x32x3  -->   5x5x8     ---> 28x28x8
conv_out1 = maxpool(conv_out1, 2)                                           # 28x28x8  -->   Max(2)    ---> 14x14x8

conv_out2 = relu(convolution(conv_out1, filter2_weights) + filter2_bias)    # 14x14x8  -->   5x5x16    ---> 10x10x16
conv_out2 = maxpool(conv_out2, 2)                                           # 10x10x16 -->   5x5x16    ---> 5x5x16
############################ Flatten Data for Conv - > Linear Operation ############################
flatten_dim= 1
for dim in conv_out2.shape:
    flatten_dim *= dim
flattened_out = conv_out2.reshape(-1, flatten_dim)                          # 5x5x16  ---   -----     ---> 1x400

 ############################ Linear Layers ############################

linear1_out = relu(linear( flattened_out,linear1_weights, linear1_bias))    # 1x400  ---  400x128  ---> 1x128 (Wx+b)
linear2_out = relu(linear( linear1_out,linear2_weights, linear2_bias))      # 1x128  ---  128x64  ---> 1x64 (Wx+b)
linear3_out = linear( linear2_out,linear3_weights, linear3_bias)            # 1x64   ---  164x10  ---> 1x10 (Wx+b)

        #Now Apply Argmax to find the Probability of the image being of a particular class (0-9) 
        # (i.e) pick the max probab returned from the linear2_out

 ############################ Output Inference ############################

out = softmax(linear3_out)
class_output = np.argmax(out)
print(class_output)


4
